In [1]:
import re

<link href="style.css" rel="stylesheet"></link>
<main>
<article class="day-desc"><h2>--- Day 16: The Floor Will Be Lava ---</h2><p>With the beam of light completely focused <em>somewhere</em>, the reindeer leads you deeper still into the Lava Production Facility. At some point, you realize that the steel facility walls have been replaced with cave, and the doorways are just cave, and the floor is cave, and you're pretty sure this is actually just a giant cave.</p>
<p>Finally, as you approach what must be the heart of the mountain, you see a bright light in a cavern up ahead. There, you discover that the <span title="Not anymore, there's a blanket!">beam</span> of light you so carefully focused is emerging from the cavern wall closest to the facility and pouring all of its energy into a contraption on the opposite side.</p>
<p>Upon closer inspection, the contraption appears to be a flat, two-dimensional square grid containing <em>empty space</em> (<code>.</code>), <em>mirrors</em> (<code>/</code> and <code>\</code>), and <em>splitters</em> (<code>|</code> and <code>-</code>).</p>
<p>The contraption is aligned so that most of the beam bounces around the grid, but each tile on the grid converts some of the beam's light into <em>heat</em> to melt the rock in the cavern.</p>
<p>You note the layout of the contraption (your puzzle input). For example:</p>
<pre><code>.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
</code></pre>
<p>The beam enters in the top-left corner from the left and heading to the <em>right</em>. Then, its behavior depends on what it encounters as it moves:</p>
<ul>
<li>If the beam encounters <em>empty space</em> (<code>.</code>), it continues in the same direction.</li>
<li>If the beam encounters a <em>mirror</em> (<code>/</code> or <code>\</code>), the beam is <em>reflected</em> 90 degrees depending on the angle of the mirror. For instance, a rightward-moving beam that encounters a <code>/</code> mirror would continue <em>upward</em> in the mirror's column, while a rightward-moving beam that encounters a <code>\</code> mirror would continue <em>downward</em> from the mirror's column.</li>
<li>If the beam encounters the <em>pointy end of a splitter</em> (<code>|</code> or <code>-</code>), the beam passes through the splitter as if the splitter were <em>empty space</em>. For instance, a rightward-moving beam that encounters a <code>-</code> splitter would continue in the same direction.</li>
<li>If the beam encounters the <em>flat side of a splitter</em> (<code>|</code> or <code>-</code>), the beam is <em>split into two beams</em> going in each of the two directions the splitter's pointy ends are pointing. For instance, a rightward-moving beam that encounters a <code>|</code> splitter would split into two beams: one that continues <em>upward</em> from the splitter's column and one that continues <em>downward</em> from the splitter's column.</li>
</ul>
<p>Beams do not interact with other beams; a tile can have many beams passing through it at the same time. A tile is <em>energized</em> if that tile has at least one beam pass through it, reflect in it, or split in it.</p>
<p>In the above example, here is how the beam of light bounces around the contraption:</p>
<pre><code>&gt;|&lt;&lt;&lt;\....
|v-.\^....
.v...|-&gt;&gt;&gt;
.v...v^.|.
.v...v^...
.v...v^..\
.v../2\\..
&lt;-&gt;-/vv|..
.|&lt;&lt;&lt;2-|.\
.v//.|.v..
</code></pre>
<p>Beams are only shown on empty tiles; arrows indicate the direction of the beams. If a tile contains beams moving in multiple directions, the number of distinct directions is shown instead. Here is the same diagram but instead only showing whether a tile is <em>energized</em> (<code>#</code>) or not (<code>.</code>):</p>
<pre><code>######....
.#...#....
.#...#####
.#...##...
.#...##...
.#...##...
.#..####..
########..
.#######..
.#...#.#..
</code></pre>
<p>Ultimately, in this example, <code><em>46</em></code> tiles become <em>energized</em>.</p>
<p>The light isn't energizing enough tiles to produce lava; to debug the contraption, you need to start by analyzing the current situation. With the beam starting in the top-left heading right, <em>how many tiles end up being energized?</em></p>
</article>

</main>


In [70]:
from collections import deque


layout = r"""
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
"""


def parse(layout: str) -> list[str]:
    return layout.strip().splitlines()


def energize(layout: list[str], start_position=(0, 0), start_velocity=(0, 1)) -> int:
    rows, cols = len(layout), len(layout[0])
    energized_tiles = set()
    processed = set()
    queue = deque([(start_position, start_velocity)])
    while queue:
        (p_r, p_c), (v_r, v_c) = queue.popleft()
        if not (0 <= p_r < rows and 0 <= p_c < cols):
            continue
        if ((p_r, p_c), (v_r, v_c)) in processed:
            continue
        processed.add(((p_r, p_c), (v_r, v_c)))
        energized_tiles.add((p_r, p_c))

        if layout[p_r][p_c] == ".":
            queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
        elif layout[p_r][p_c] == "|":
            if (v_r, v_c) == (0, 1):
                (v_r, v_c) = (1, 0)
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
                (v_r, v_c) = (-1, 0)
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
            elif (v_r, v_c) == (0, -1):
                (v_r, v_c) = (1, 0)
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
                (v_r, v_c) = (-1, 0)
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
            elif (v_r, v_c) == (1, 0):
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
            else:  # (v_r, v_c) == (-1, 0)
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
        elif layout[p_r][p_c] == "-":
            if (v_r, v_c) == (0, 1):
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
            elif (v_r, v_c) == (0, -1):
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
            elif (v_r, v_c) == (1, 0):
                (v_r, v_c) = (0, 1)
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
                (v_r, v_c) = (0, -1)
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
            else:  # (v_r, v_c) == (-1, 0)
                (v_r, v_c) = (0, 1)
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
                (v_r, v_c) = (0, -1)
                queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
        elif layout[p_r][p_c] == "/":
            if (v_r, v_c) == (0, 1):
                (v_r, v_c) = (-1, 0)
            elif (v_r, v_c) == (0, -1):
                (v_r, v_c) = (1, 0)
            elif (v_r, v_c) == (1, 0):
                (v_r, v_c) = (0, -1)
            else:  # (v_r, v_c) == (-1, 0)
                (v_r, v_c) = (0, 1)
            queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))
        else:  # layout[p_r][p_c] == "\":
            if (v_r, v_c) == (0, 1):
                (v_r, v_c) = (1, 0)
            elif (v_r, v_c) == (0, -1):
                (v_r, v_c) = (-1, 0)
            elif (v_r, v_c) == (1, 0):
                (v_r, v_c) = (0, 1)
            else:  # (v_r, v_c) == (-1, 0)
                (v_r, v_c) = (0, -1)
            queue.append(((p_r + v_r, p_c + v_c), (v_r, v_c)))

    return len(energized_tiles)


assert energize(parse(layout)) == 46

In [71]:
with open("./input/day16.txt") as f:
    print(energize(parse(f.read())))

7951


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>7951</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you try to work out what might be wrong, the reindeer tugs on your shirt and leads you to a nearby control panel. There, a collection of buttons lets you align the contraption so that the beam enters from <em>any edge tile</em> and heading away from that edge. (You can choose either of two directions for the beam if it starts on a corner; for instance, if the beam starts in the bottom-right corner, it can start heading either left or upward.)</p>
<p>So, the beam could start on any tile in the top row (heading downward), any tile in the bottom row (heading upward), any tile in the leftmost column (heading right), or any tile in the rightmost column (heading left). To produce lava, you need to find the configuration that <em>energizes as many tiles as possible</em>.</p>
<p>In the above example, this can be achieved by starting the beam in the fourth tile from the left in the top row:</p>
<pre><code>.|&lt;2&lt;\....
|v-v\^....
.v.v.|-&gt;&gt;&gt;
.v.v.v^.|.
.v.v.v^...
.v.v.v^..\
.v.v/2\\..
&lt;-2-/vv|..
.|&lt;&lt;&lt;2-|.\
.v//.|.v..
</code></pre>
<p>Using this configuration, <code><em>51</em></code> tiles are energized:</p>
<pre><code>.#####....
.#.#.#....
.#.#.#####
.#.#.##...
.#.#.##...
.#.#.##...
.#.#####..
########..
.#######..
.#...#.#..
</code></pre>
<p>Find the initial beam configuration that energizes the largest number of tiles; <em>how many tiles are energized in that configuration?</em></p>
</article>

</main>


In [83]:
layout = r"""
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
"""


def optimize_energizing(layout: str):
    rows, cols = len(layout), len(layout[0])
    energie = 0
    p_r, p_c = 0, 0
    v_r, v_c = 0, 1
    for r in range(rows):
        energie = max(
            energie,
            energize(layout, (r, 0), (0, 1)),
            energize(layout, (r, cols - 1), (0, -1)),
        )
    for c in range(cols):
        energie = max(
            energie,
            energize(layout, (0, c), (1, 0)),
            energize(layout, (rows - 1, c), (0, -1)),
        )
    return energie


assert optimize_energizing(parse(layout)) == 51

In [84]:
with open("./input/day16.txt") as f:
    print(optimize_energizing(parse(f.read())))

8148


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>8148</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2023">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="16/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22The+Floor+Will+Be+Lava%22+%2D+Day+16+%2D+Advent+of+Code+2023&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F16&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22The+Floor+Will+Be+Lava%22+%2D+Day+16+%2D+Advent+of+Code+2023+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F16';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
